In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tflearn

# Load and Process the Data

In [12]:
data = np.loadtxt('data/data.csv', delimiter=',')

The last 13 rows of the data represent my output

In [13]:
X = np.transpose(data[:-13,:])
Y = np.transpose(data[-13:,:])

Every channel has a background of around ~150. We don't want to factor that in. Output should always be positive, so these have been scaled in the range $\left[0,1\right]$.

In [14]:
inscale = MinMaxScaler()
outscale = MinMaxScaler()

In [15]:
xs = inscale.fit_transform(X)
ys = outscale.fit_transform(Y)

Split the data into **train**, **test**, and **val**idation segments.

The data are in blocks of 20 measurements for each sample. Since the samples should be independent of each other, we can randomly select from each of the 7 samples. I have elected for a 70/15/15 train/test/validate split. 

In [16]:
# Choose 6 random numbers from 1-20.
c = np.random.choice(range(20), 6, replace=False)

# Make these the indexes we will use for our test/validation
while c.shape[0] <= (xs.shape[0] - 6) * 0.3:
    c = np.concatenate((c, c[-6:]+20))

In [17]:
xs_test = np.array([xs[i, :] for i in c[::2]])
ys_test = np.array([ys[i, :] for i in c[::2]])
xs_val = np.array([xs[i, :] for i in c[1::2]])
ys_val = np.array([ys[i, :] for i in c[1::2]])
xs_train = np.delete(xs, c, 0)
ys_train = np.delete(ys, c, 0)
print(xs_test.shape)
print(ys_test.shape)
print(xs_val.shape)
print(ys_val.shape)
print(xs_train.shape)
print(ys_train.shape)

(21, 11725)
(21, 13)
(21, 11725)
(21, 13)
(98, 11725)
(98, 13)


We now have 98 training records, 21 testing records, and 21 validation records. Each record is a vector of 11725 input values and 13 output values.

In [18]:
ys_train.shape

(98, 13)

# Classifier

In [19]:
tf.__version__

'1.1.0'

In [20]:
from tflearn.datasets import titanic
titanic.download_dataset('titanic_dataset.csv')

Succesfully downloaded titanic_dataset.csv 82865 bytes.


'./titanic_dataset.csv'

In [21]:
from tflearn.data_utils import load_csv

In [22]:
data, labels = load_csv('data/tflearn_data.csv', target_column=0, categorical_labels=True, n_classes=7)

In [23]:
len(data[0])

11725

## Build a Deep Neural Network

In [ ]:
net = tflearn.input_data(shape=[None, 11725])
net = tflearn.fully_connected(net, 20000)
net = tflearn.fully_connected(net, 20000)
net = tflearn.fully_connected(net, 20000)
net = tflearn.fully_connected(net, 20000)
net = tflearn.fully_connected(net, 20000)
net = tflearn.fully_connected(net, 13, activation='softmax')
net = tflearn.regression(net)

In [ ]:
model = tflearn.DNN(net, tensorboard_verbose=3, tensorboard_dir='./')
model.fit(xs_train, ys_train, validation_set=(xs_val, ys_val),
          n_epoch=10, batch_size=16, show_metric=True)

In [ ]:
model.predict_label(np.array(data)[:]).shape

In [ ]:
plt.hist(model.predict(np.array(data)))